<a href="https://colab.research.google.com/github/JoaoDaross/CursoAwari/blob/main/Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importando dados preprocessados

In [2]:
import pandas as pd

treino = pd.read_csv('https://raw.githubusercontent.com/JoaoDaross/CursoAwari/main/treino_preprocessado.csv')
teste = pd.read_csv('https://raw.githubusercontent.com/JoaoDaross/CursoAwari/main/teste_preprocessado.csv')

In [3]:
# dividindo os dados de treino e teste (X e y)

X_train = treino.drop('precos', axis=1)
y_train = treino['precos']
X_test = teste.drop('precos', axis=1)
y_test = teste['precos']

Criando um modelo de base para comparar os resultados (baseline)

In [4]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.012382909039824086

In [5]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math 

lr = LinearRegression()
lr.fit(X_train, y_train)
lr_y_pred = lr.predict(X_test)

mse = mean_squared_error(lr_y_pred, y_test)
mae = mean_absolute_error(lr_y_pred, y_test)

# olhando diferentes métricas
print('mse:', mse)
print('rmse:', math.sqrt(mse))
print('mae:', mae)

mse: 2.3608600165870186
rmse: 1.5365090356346813
mae: 0.9404555759013817


In [6]:
from sklearn.dummy import DummyRegressor

reg = DummyRegressor(strategy='mean')
reg.fit(X_train, y_train)
reg.score(X_test, y_test)

-6.654056328159186e-08

Comparando diferentes modelos de regressão (estimadores)

In [7]:
from sklearn.linear_model import RidgeCV, Lasso, ElasticNet, LassoLars, HuberRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

In [8]:
reg_list = [RidgeCV(),
            SVR(),
            KNeighborsRegressor(),
            RandomForestRegressor(),
            AdaBoostRegressor(),
            GradientBoostingRegressor(),
            MLPRegressor()
            ]

In [9]:
from sklearn.model_selection import cross_val_score
import numpy as np

for reg in reg_list:
    print(f'Treinando o modelo {reg.__class__.__name__}')
    reg.fit(X_train, y_train)
    
    train_score = reg.score(X_train, y_train)
    cv_scores = cross_val_score(reg, X_train, y_train)
    test_score = reg.score(X_test, y_test)
    
    print(f"R² treino: {train_score}")
    print(f"R² validação : {np.mean(cv_scores):.2f} +- {np.std(cv_scores):.2f}")
    print(f"R² teste: {test_score}")
    print('-'*70)

Treinando o modelo RidgeCV
R² treino: 0.04944626631224058
R² validação : 0.02 +- 0.03
R² teste: 0.01437775664853236
----------------------------------------------------------------------
Treinando o modelo SVR
R² treino: -0.09404811596340656
R² validação : -0.10 +- 0.05
R² teste: -0.10290984958936789
----------------------------------------------------------------------
Treinando o modelo KNeighborsRegressor
R² treino: 0.27042583142122545
R² validação : -0.02 +- 0.11
R² teste: 0.11538007828409234
----------------------------------------------------------------------
Treinando o modelo RandomForestRegressor
R² treino: 0.46995271338920713
R² validação : -0.17 +- 0.17
R² teste: 0.08589069513071335
----------------------------------------------------------------------
Treinando o modelo AdaBoostRegressor
R² treino: 0.014366078746962163
R² validação : -0.06 +- 0.09
R² teste: -0.0145990054773244
----------------------------------------------------------------------
Treinando o modelo Gradien

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


R² treino: 0.10354119899901326
R² validação : 0.05 +- 0.02
R² teste: 0.06325211272552367
----------------------------------------------------------------------


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [10]:
# Bonus: testando com todos os regressores do sklearn
from sklearn.utils import all_estimators

estimators = all_estimators(type_filter='regressor')

relatorio = {'nome':[],
             'train_score':[],
             'cv_scores_mean':[],
             'test_score':[],
             'estimador':[]
             }

ignore_list = ['IsotonicRegression',
 'MultiOutputRegressor',
 'ElasticNet',
 'MultiTaskElasticNet',
 'MultiTaskElasticNetCV',
 'MultiTaskLasso',
 'MultiTaskLassoCV',
 'RadiusNeighborsRegressor',
 'RegressorChain',
 'StackingRegressor',
 'VotingRegressor']

In [11]:
for name, RegressorClass in estimators:
    if name not in ignore_list:
        print(f'Treinando o modelo {name}')
        reg = RegressorClass()
        reg.fit(X_train, y_train)

        train_score = reg.score(X_train, y_train)
        cv_scores = cross_val_score(reg, X_train, y_train)
        test_score = reg.score(X_test, y_test)

        print(f"R² treino: {train_score}")
        print(f"R² validação: {np.mean(cv_scores):.2f} +- {np.std(cv_scores):.2f}")
        print(f"R² teste: {test_score}")
        print('-'*70)

        relatorio['nome'].append(name)
        relatorio['train_score'].append(train_score)
        relatorio['cv_scores_mean'].append(np.mean(cv_scores))
        relatorio['test_score'].append(test_score)
        relatorio['estimador'].append(reg)

Treinando o modelo ARDRegression
R² treino: 0.04790389539098927
R² validação: 0.02 +- 0.03
R² teste: 0.016736502060411773
----------------------------------------------------------------------
Treinando o modelo AdaBoostRegressor
R² treino: 0.06732403972449796
R² validação: -0.04 +- 0.11
R² teste: 0.05756186145354659
----------------------------------------------------------------------
Treinando o modelo BaggingRegressor
R² treino: 0.4443941454407897
R² validação: -0.23 +- 0.17
R² teste: 0.13217803335765754
----------------------------------------------------------------------
Treinando o modelo BayesianRidge
R² treino: 0.04854050729586945
R² validação: 0.02 +- 0.03
R² teste: 0.018194167562858632
----------------------------------------------------------------------
Treinando o modelo CCA
R² treino: -0.21379029353079537
R² validação: -0.34 +- 0.30
R² teste: -0.31615226221832393
----------------------------------------------------------------------
Treinando o modelo DecisionTreeRegres

/usr/local/lib/python3.6/dist-packages/sklearn/cross_decomposition/_pls.py:321: UserWarning: Y residual constant at iteration 1
  warnings.warn('Y residual constant at iteration %s' % k)
/usr/local/lib/python3.6/dist-packages/sklearn/cross_decomposition/_pls.py:321: UserWarning: Y residual constant at iteration 1
  warnings.warn('Y residual constant at iteration %s' % k)
/usr/local/lib/python3.6/dist-packages/sklearn/cross_decomposition/_pls.py:321: UserWarning: Y residual constant at iteration 1
  warnings.warn('Y residual constant at iteration %s' % k)
/usr/local/lib/python3.6/dist-packages/sklearn/cross_decomposition/_pls.py:321: UserWarning: Y residual constant at iteration 1
  warnings.warn('Y residual constant at iteration %s' % k)
/usr/local/lib/python3.6/dist-packages/sklearn/cross_decomposition/_pls.py:321: UserWarning: Y residual constant at iteration 1
  warnings.warn('Y residual constant at iteration %s' % k)
/usr/local/lib/python3.6/dist-packages/sklearn/cross_decompositio

R² treino: 0.04913529782756065
R² validação: 0.02 +- 0.03
R² teste: 0.01637451302209736
----------------------------------------------------------------------
Treinando o modelo ExtraTreeRegressor
R² treino: 0.5122363073776075
R² validação: -0.43 +- 0.26
R² teste: -0.07704635694018291
----------------------------------------------------------------------
Treinando o modelo ExtraTreesRegressor
R² treino: 0.5122363073776075
R² validação: -0.35 +- 0.23
R² teste: 0.040247565327446444
----------------------------------------------------------------------
Treinando o modelo GaussianProcessRegressor
R² treino: 0.2585732312362814
R² validação: -18733.62 +- 34347.86
R² teste: -286.2491387863656
----------------------------------------------------------------------
Treinando o modelo GradientBoostingRegressor
R² treino: 0.31399877055840075
R² validação: -0.02 +- 0.09
R² teste: 0.15053896343499185
----------------------------------------------------------------------
Treinando o modelo HistGradie

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

R² treino: 0.09893376857642011
R² validação: 0.03 +- 0.05
R² teste: 0.055855267491657035
----------------------------------------------------------------------
Treinando o modelo NuSVR
R² treino: -0.07839961673318441
R² validação: -0.09 +- 0.04
R² teste: -0.08836170672975441
----------------------------------------------------------------------
Treinando o modelo OrthogonalMatchingPursuit
R² treino: 0.03198500919470226
R² validação: 0.02 +- 0.02
R² teste: 0.009818497146758864
----------------------------------------------------------------------
Treinando o modelo OrthogonalMatchingPursuitCV
R² treino: 0.04770167140320003
R² validação: 0.01 +- 0.03
R² teste: 0.01682990050748745
----------------------------------------------------------------------
Treinando o modelo PLSCanonical
R² treino: -0.8959468515240194
R² validação: -1.10 +- 0.68
R² teste: -0.9167296764972055
----------------------------------------------------------------------
Treinando o modelo PLSRegression
R² treino: 0.0476

/usr/local/lib/python3.6/dist-packages/sklearn/cross_decomposition/_pls.py:321: UserWarning: Y residual constant at iteration 1
  warnings.warn('Y residual constant at iteration %s' % k)
/usr/local/lib/python3.6/dist-packages/sklearn/cross_decomposition/_pls.py:321: UserWarning: Y residual constant at iteration 1
  warnings.warn('Y residual constant at iteration %s' % k)
/usr/local/lib/python3.6/dist-packages/sklearn/cross_decomposition/_pls.py:321: UserWarning: Y residual constant at iteration 1
  warnings.warn('Y residual constant at iteration %s' % k)
/usr/local/lib/python3.6/dist-packages/sklearn/cross_decomposition/_pls.py:321: UserWarning: Y residual constant at iteration 1
  warnings.warn('Y residual constant at iteration %s' % k)
/usr/local/lib/python3.6/dist-packages/sklearn/cross_decomposition/_pls.py:321: UserWarning: Y residual constant at iteration 1
  warnings.warn('Y residual constant at iteration %s' % k)
/usr/local/lib/python3.6/dist-packages/sklearn/cross_decompositio

R² treino: -0.21486937364747472
R² validação: -0.22 +- 0.07
R² teste: -0.2021819243508358
----------------------------------------------------------------------
Treinando o modelo RandomForestRegressor
R² treino: 0.47450544453722177
R² validação: -0.18 +- 0.17
R² teste: 0.10067571748288763
----------------------------------------------------------------------
Treinando o modelo Ridge
R² treino: 0.04944626631224058
R² validação: 0.02 +- 0.03
R² teste: 0.014377756648527916
----------------------------------------------------------------------
Treinando o modelo RidgeCV
R² treino: 0.04944626631224058
R² validação: 0.02 +- 0.03
R² teste: 0.01437775664853236
----------------------------------------------------------------------
Treinando o modelo SGDRegressor
R² treino: 0.02429826738959251
R² validação: -0.02 +- 0.04
R² teste: -0.017193811350596278
----------------------------------------------------------------------
Treinando o modelo SVR
R² treino: -0.09404811596340656
R² validação: -0.1

In [12]:
relatorio = pd.DataFrame(relatorio).sort_values(by='cv_scores_mean', ascending=False)
relatorio.head(3)

,nome,train_score,cv_scores_mean,test_score,estimador
25,MLPRegressor,0.098934,0.032778,0.055855,"MLPRegressor(activation='relu', alpha=0.0001, ..."
39,TransformedTargetRegressor,0.049428,0.019696,0.012383,"TransformedTargetRegressor(check_inverse=True,..."
23,LinearRegression,0.049428,0.019696,0.012383,"LinearRegression(copy_X=True, fit_intercept=Tr..."



Combinando os melhores estimadores usando stacking

In [13]:
from sklearn.ensemble import StackingRegressor

top_3_regs = relatorio[['nome', 'estimador']].values[:3]

reg = StackingRegressor(
    estimators = top_3_regs
)

reg.fit(X_train, y_train)

train_score = reg.score(X_train, y_train)
cv_scores = cross_val_score(reg, X_train, y_train)
test_score = reg.score(X_test, y_test)

print(f"R² treino: {train_score}")
print(f"R² validação: {np.mean(cv_scores):.2f} +- {np.std(cv_scores):.2f}")
print(f"R² teste: {test_score}")
print('-'*70)

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


R² treino: 0.07388732915430785
R² validação: 0.03 +- 0.02
R² teste: 0.03662241380778575
----------------------------------------------------------------------


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [14]:
reg

StackingRegressor(cv=None,
                  estimators=array([['MLPRegressor',
        MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0...
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)],
       ['TransformedTargetRegressor',
        TransformedTargetRegressor(check_inverse=True, func=None, inverse_func=None,
                           regressor=None, transformer=None)],
       ['LinearRegression',
        LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)]],
      dtype=object),
                  final_estimator=None, n_jobs=None, passthrough=False,
                  verbose=0)

Salvando o modelo

In [15]:
import pickle

pickle.dump(reg, open('modelo.pkl', 'wb'), protocol=4)

In [16]:
# resetando o notebook para garantir que estamos carregando o modelo salvo
%reset -f

In [17]:
import pickle

reg = pickle.load(open('modelo.pkl', 'rb'))

In [18]:
reg

StackingRegressor(cv=None,
                  estimators=array([['MLPRegressor',
        MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(100,), learning_rate='constant',
             learning_rate_init=0.001, max_fun=15000, max_iter=200,
             momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
             power_t=0...
             tol=0.0001, validation_fraction=0.1, verbose=False,
             warm_start=False)],
       ['TransformedTargetRegressor',
        TransformedTargetRegressor(check_inverse=True, func=None, inverse_func=None,
                           regressor=None, transformer=None)],
       ['LinearRegression',
        LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)]],
      dtype=object),
                  final_estimator=None, n_jobs=None, passthrough=False,
                  verbose=0)

In [19]:
import numpy as np

dados_novos = [0, 0, 0, 1, np.log1p(2), np.log1p(120)]

np.expm1(reg.predict([dados_novos]))

array([3861.01518221])